# Analyse plancapaciteit woningen

In deze notebook zetten we cijfers op een rij om de woningbouwplannen van Nederlandse gemeenten in kaart te kunnen brengen. Beschikbare datasets zijn:

1. [Plancapaciteit per gemeente.](https://docs.google.com/spreadsheets/d/1v5jXy01Xfx7tZHE5Wna7StYFtER3Xnus/edit?usp=sharing&ouid=113330074508532941534&rtpof=true&sd=true) Deze dataset is samengesteld door Data Bewijst en bevat enkele mitsen en maren:
* De meeste provincies werken met het onderscheid harde en zachte plannen en hanteren de fasering 2020-2024, 2025-2029, 2030 en verder.
* Limburg hanteert de fasering 2021-2023, 2024 en verder en maakt wel onderscheid in hard en zacht.
* Zuid-Holland gooit alle harde en zachte plannen op een hoop en hanteert 1 fase: 2021-2030
* Groningen maakt onderscheid tussen hard en zacht, maar hanteert de periode 2020-2029
* Drenthe gooit alles op 1 hoop en het is onduidelijk welke periode het hanteert. Daarnaast lijken de cijfers niet te kloppen. Dat wordt nog gechecked.
* Friesland maakt onderscheid tussen hard en zacht, maar geeft voor beide 1 tijdsperiode: 2021-2030.
* Overijssel maakt wel gebruik van de fasering voor harde plannen, maar alle zachte plannen krijgen 1 tijdsfasering, namelijk 2021-2030.
* De gemeente Haaren is in 2021 opgegaan in Vught, Boxtel, Oisterwijk en Tilburg. De vraag is: wat doen we daarmee? Alle vier gemeenten een kwart van Haaren erbij? Waarschijnlijk kunnen we het beste de indeling van 2020 gebruiken.

2. [CBS woningvoorraad per gemeente](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=81955NED&_theme=275). Deze data is gebaseerd op de BAG en bestrijkt de periode 2012-2020.

3. Plannen van [De Nieuwe Kaart](https://nieuwekaartnl.nl/). Deze kaart is alleen niet volledig (een aantal provincies ontbreekt), toont gemengde data (polygonen en punten die als polygoon worden weergegeven) en is niet helemaal *up to date* meer. 

4. Eigendomsgegevens van de RVO. Dit zijn niet-openbare data die we eigenlijk niet hadden mogen hebben. Per perceel wordt het type eigendom gegeven (woningbouwcorporatie, bouw- of projectontwikkelaar, publiek gemeente, publiek rijk, etc.) Deze data zijn van 2019 en een exacte datum ontbreekt, dus we moeten ervan uitgaan dat ze op 1 januari 2019 zijn gemaakt. 

5. Data van de [nieuwbouwmonitor](https://geodienst.xyz/nieuwbouwmonitor/) van de Geodienst van de RUG. Dit is gebaseerd op BAG data en geverifieerd met data van het CBS. Toont de woningbouw per gemeente sinds 2014 en rekent ook de doorlooptijd van vergunning tot oplevering uit. In veel opzichten lijkt deze op de dataset van het CSB.

6. [De Basisregistratie Kadaster](https://www.pdok.nl/introductie/-/article/basisregistratie-kadaster-brk-) voor percelen en kadastrale grenzen. Uit het register zijn mutaties te halen die mogelijk interessant zijn voor de analyse. 

In [ ]:
import pandas as pd
import datetime as dt
import glob
import xlrd
import geopandas as gpd
import plotly.express as px
import config

## Plancapaciteit

Eerst moeten we de data opschonen en normaliseren zodat we ze later makkelijker kunnen combineren met andere datasets. We hebben daarvoor eerst een lijst met gemeentecodes nodig.

In [ ]:
PATH = config.PATH
PATH_GEMEENTEN = config.PATH_GEMEENTEN

In [ ]:
gemeenten = []

for file in glob.glob(PATH_GEMEENTEN + 'Gemeenten*'):
    cbs = pd.read_excel(file, usecols=['Gemeentenaam', 'Gemeentecode'])
    gemeenten.append(cbs)
    
gemcode = pd.concat(gemeenten)
gemcode = gemcode.drop_duplicates('Gemeentenaam', keep='last').sort_values(by='Gemeentenaam')

# Convert Gemeentecode into string, so we can manipulate it easier.

gemcode['Gemeentecode'] = gemcode['Gemeentecode'].astype(str)

# Add column with Gemcode: we need this later on for merging datasets

gemcode['Gemcode'] = gemcode['Gemeentecode'].apply(lambda s: 'GM00' + s if len(s) < 3 else ('GM0' + s if len(s) == 3 else 'GM' + s))

gemcode.head()

Vervolgens importeren we de plancapaciteit per gemeente en koppelen die aan de gemeentecodes. Ook hier moet nog het e.e.a. worden opgeschoond. Let op: Limburg, Friesland en Overijssel hebben iets afwijkende termijnen en Drenthe klopt nog niet helemaal, maar dat maakt op het totaal niet uit. 

In [ ]:
plan = pd.read_excel(PATH + 'plancapaciteit_databewijst.xlsx', sheet_name='plancapaciteit_nl')

# We moeten een aantal gemeentenamen veranderen, anders gaat het koppelen niet goed. 

names_to_replace = {'Bergen': 'Bergen (NH.)',
                    'Nuenen c.a.': 'Nuenen, Gerwen en Nederwetten',
                    'NederBetuwe': 'Neder-Betuwe', 
                    'Oude Ijsselstreek': 'Oude IJsselstreek', 
                    'Súdwest Fryslân': 'Súdwest-Fryslân',
                    'Bergen (L)': 'Bergen (L.)',
                    'Eemsdelta': 'Delfzijl'}

plan['gemeente'] = plan['gemeente'].replace(names_to_replace)

# Merge de twee bestanden

plannen = pd.merge(plan, 
                       gemcode[['Gemeentenaam', 'Gemcode']], 
                       left_on='gemeente', 
                       right_on='Gemeentenaam', 
                       how='left')

# En nog even wat netter maken

plannen = plannen.rename(columns={'Gemcode': 'gemeentecode',
                                          'htotaal': 'harde_plannen_totaal',
                                          'ztotaal': 'zachte_plannen_totaal',
                                          'totaal': 'plannen_totaal'})

# En laten we even alleen de totalen gebruiken en ca tien jaar vooruit kijken.

plannen = plannen[['gemeente', 
                           'gemeentecode', 
                           'provincie',
                           'harde_plannen_totaal',
                           'zachte_plannen_totaal',
                           'plannen_totaal']]

# Tot slot nog even het datatype van de totalen veranderen van float naar int

cols = ['harde_plannen_totaal', 'zachte_plannen_totaal', 'plannen_totaal']

plannen[cols] = plannen[cols].fillna(0).astype('int64')

plannen.head()

Laten we wat statistieken eruit halen. Eerst de totalen. Let op, harde en zachte plannen bij elkaar leveren een ander getal op dan het totaal. Dat komt doordat sommige provincies alleen maar totalen hebben opgegeven en niet uitsplitsen naar hard en zacht.

In [ ]:
print('Er staan ' + str(plannen.harde_plannen_totaal.sum()) + ' huizen in de harde plannen vanaf 2020.\n' + \
      'Er staan ' + str(plannen.zachte_plannen_totaal.sum()) + ' huizen in de zachte plannen van 2020.\n' + \
      'En dat zijn er dus ' + str(plannen.plannen_totaal.sum()) + ' in totaal')

Dan wat lijstjes.

In [ ]:
# Meeste woningbouw (plannen totaal)

plannen[['gemeente', 'plannen_totaal']].sort_values('plannen_totaal', ascending=False)[:10]

In [ ]:
# Meeste woningbouw (harde plannen, let op: hier ontbreken dus gemeenten):

plannen[['gemeente', 'harde_plannen_totaal']].sort_values('harde_plannen_totaal', ascending=False)[:10]

In [ ]:
# Meeste woningbouw (zachte plannen, let op: hier ontbreken dus gemeenten)

plannen[['gemeente', 'zachte_plannen_totaal']].sort_values('zachte_plannen_totaal', ascending=False)[:10]


In [ ]:
# En even nog naar provincies kijken (totale plannen), maar waarschijnlijk levert dat weinig verrassingen op

plannen.groupby('provincie')['plannen_totaal'].sum().sort_values(ascending=False)

Toch wel een paar opvallende dingen:
1. Relatief veel in Noord-Holland. Zuid-Holland is qua bevolkingsaantal de meest omvangrijke provincie (3,7 miljoen inwoners t.o. 2,8 miljoen inwoners in Noord-Holland)
2. Flevoland, daar wonen nu iets meer dan 428.000 mensen.

## CBS Data Woningvoorraad

Het is ook interessant om de geplande huizenbouw te vergelijken met wat er de afgelopen tien jaar is gebouwd en de huidige woningvoorraad. Het CBS heeft die cijfers. We moeten de woningvoorraad per gemeente importeren en de mutaties over tijd laten zien. 

In [ ]:
cbs_woning = pd.read_csv(PATH + 'cbs_woningvoorraad_gemeenten_2012-2020.csv', sep=';')
cbs_woning.head()

In [ ]:
# Ook dit even wat netter maken

cbs_woning = cbs_woning.rename(columns={'Gebruiksfunctie': 'gebruiksfunctie',
                                         'RegioS': 'gemeentecode',
                                         'Perioden': 'jaar',
                                         'BeginstandVoorraad_1': 'beginstand_woningvoorraad',
                                         'Nieuwbouw_2': 'nieuwbouw',
                                         'OverigeToevoeging_3': 'overige_toevoeging',
                                         'Sloop_4': 'sloop',
                                         'OverigeOnttrekking_5': 'overige_onttrekking', 
                                         'Correctie_6': 'correctie',
                                         'SaldoVoorraad_7': 'saldo_voorraad',
                                         'EindstandVoorraad_8': 'eindstand_voorraad'})

# Columns even numeriek maken (werkt nog niet goed, later nog even naar kijken)

cbs_woning = cbs_woning.dropna(subset=['nieuwbouw'])

cols = ['beginstand_woningvoorraad', 
        'nieuwbouw', 
        'overige_toevoeging', 
        'sloop', 
        'overige_onttrekking',
        'correctie',
        'saldo_voorraad',
        'eindstand_voorraad']

cbs_woning[cols] = cbs_woning[cols].astype('int64')

# Verander de 'jaar' column in date

cbs_woning['jaar'] = cbs_woning['jaar'].str[0:4]
cbs_woning['jaar'] = pd.to_datetime(cbs_woning.jaar, format='%Y').dt.year

cbs_woning.head()

## En nu combineren

We hebben de basale datasets op orde en kunnen nu gaan combineren. Laten we beginnen met een samenvattende tabel maken voor gemeenten met daarin:
1. gemeentecode
2. woningvooraad laagste waarde
3. woningvooraad laagste waarde - jaar
4. woningvoorraad hoogste waarde
5. woningvoorraad hoogste waarde - jaar
6. woningvoorraad procentueel verschil
7. nieuwbouw laagste waarde
8. nieuwbouw hoogste waarde
9. nieuwbouw als percentage van woningvoorraad
10. nieuwbouw som 2012-2020

In [ ]:
# Bereken mediaan, min en max

voorraad = cbs_woning.groupby(["gemeentecode"], as_index=False)['beginstand_woningvoorraad']\
.agg(['median', 'min', 'max'])

# Bereken percentage

voorraad['dif_voorraad'] = voorraad['max'] / voorraad['min'] * 100 - 100

# Rond het netjes af

voorraad['dif_voorraad'] = voorraad['dif_voorraad'].round(1)

voorraad.head()

In [ ]:
nieuwbouw = cbs_woning.groupby(["gemeentecode"], as_index=False)['nieuwbouw'].agg(['median', 'min', 'max', 'sum'])

nieuwbouw.head()

In [ ]:
# Voeg ze samen

voorraad = voorraad.rename(columns = {'median': 'voorraad_median', 
                                      'min': 'voorraad_min', 
                                      'max': 'voorraad_max'})

nieuwbouw = nieuwbouw.rename(columns = {'median': 'nieuwbouw_median', 
                                        'min': 'nieuwbouw_min', 
                                        'max': 'nieuwbouw_max',
                                        'sum': 'nieuwbouw_sum'})

df = pd.merge(voorraad, nieuwbouw, on='gemeentecode', how='left')
df.head()


In [ ]:
# Dan moeten we nog: nieuwbouw als percentage van de woningvoorraad (mediaan)...

df['nieuwbouw_perc'] = df['nieuwbouw_sum'] / df['voorraad_median'] * 100
df['nieuwbouw_perc'] = df['nieuwbouw_perc'].round(1)
df.head()

In [ ]:
# En jaar van min en max woningvoorraad

idx_max = cbs_woning.groupby(['gemeentecode'])['beginstand_woningvoorraad'].transform(max) == cbs_woning['beginstand_woningvoorraad']
idx_min = cbs_woning.groupby(['gemeentecode'])['beginstand_woningvoorraad'].transform(min) == cbs_woning['beginstand_woningvoorraad']

max_jaar = cbs_woning[idx_max]
max_jaar = max_jaar[['gemeentecode', 'jaar']]
max_jaar = max_jaar.rename(columns={'jaar':'max_jaar'})
min_jaar = cbs_woning[idx_min]
min_jaar = min_jaar[['gemeentecode', 'jaar']]
min_jaar = min_jaar.rename(columns={'jaar': 'min_jaar'})

In [ ]:
# Merge met df

df1 = pd.merge(df, max_jaar, on='gemeentecode', how='left')
cbs_woningvoorraad = pd.merge(df1, min_jaar, on='gemeentecode', how='left')
cbs_woningvoorraad.head()

In [ ]:
# And bring it all together

df = pd.merge(plannen, cbs_woningvoorraad, on='gemeentecode', how='left')
df.head()

In [ ]:
# We kunnen nog een paar kolommen toevoegen

# Zoals het absolute verschil tussen plannen (2020-2030) en nieuwbouw (2012-2020)

df['plannen_nieuwbouw_abs'] = df['plannen_totaal'] - df['nieuwbouw_sum']

# Nieuwbouw is twee jaar korter, dus we zouden daar heel ruw voor kunnen compenseren

df['plannen_nieuwbouw_abs_comp'] = (df['plannen_totaal'] * 0.8) - df['nieuwbouw_sum']

# En het relatieve verschil

df['plannen_nieuwbouw_rel'] = df['plannen_totaal'] / df['nieuwbouw_sum'] * 100 - 100
df['plannen_nieuwbouw_rel'] = df['plannen_nieuwbouw_rel'].round(1)

# En het relatieve verschil, gecompenseerd

df['plannen_nieuwbouw_rel_comp'] = ((df['plannen_totaal'] * 0.8)/ df['nieuwbouw_sum']) * 100 - 100
df['plannen_nieuwbouw_rel_comp'] = df['plannen_nieuwbouw_rel_comp'].round(1)

# En verschil tussen max woningvoorraad en plannen, absoluut en relatief

df['voorraad_plannen_abs'] = df['voorraad_max'] - df['plannen_totaal']
df['voorraad_plannen_rel'] = (df['plannen_totaal'] / df['voorraad_max']) * 100
df['voorraad_plannen_rel'] = df['voorraad_plannen_rel'].round(1)


In [ ]:
df.head()

Bovenstaande berekening lees je dus als volgt:
1. plannen_nieuwbouw_abs: 854 betekent dat er 854 meer huizen staan gepland dan er in 2012-2020 zijn gebouwd
2. plannen_nieuwbouw_abs_comp: als we uitgaan van 2010-2020 (iets eerlijker vergelijking met alle mitsen en maren) dan komt dit uit op 636 meer. 
3. plannen_nieuwbouw_rel: 78,3 procent betekent dat er 78,3 procent meer huizen gepland zijn dan er in 2012-2020 is gebouwd. 
4. plannen_nieuwbouw_rel_com: 48 procent betekent dus 48 procent meer huizen gepland dan er gebouwd zijn in 2010 (schatting) tot en met 2020.
5. voorraad_plannen_abs: het verschil tussen maximale woningvoorraad en de plannen. zegt niet zo gek veel
6. voorraad_plannen_rel: 6,9 is dus het percentage van geplande woningen t.o.v. de woningvoorraad. De woningvoorraad breidt dus, als de plannen doorgaan, met 6,9 procent uit. 

## En dan nu wat inzichten eruit halen

Met deze scatterplot kun je inzoomen en zien welke gemeente het is door erover te hoveren. Amsterdam is een behoorlijke uitschieter, dus die vertekent. Je kunt makkelijk inzoomen. Het meest interessant zijn de gemeenten linksboven en rechtsonder, dus boven en onder een denkbeeldige trendlijn. Erboven is veel gebouwd, wat minder in de planning. Eronder is weinig gebouwd, juist veel in de planning. 
1. Je ziet dan dat deze steden relatief veel gaan bouwen: Zaanstad, Haarlemmermeer, Alkmaar, Lelystad, Haarlem, Alkmaar, Heerhugowaard, Amstelveen, Beverwijk. Oftewel: veel rond Amsterdam en in ieder geval Noord-Holland.
2. Steden als Eindhoven, Tilburg, Breda, Den Bosch en Nijmegen hadden al veel nieuwbouw en gaan redelijk daarmee door. 
3. Kleinere steden zoals Heumen, Duiven en Waterland gaan drie keer zoveel bouwen als ze de laatste jaren hebben gedaan. 

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.models import *
from bokeh.palettes import plasma
from bokeh.plotting import figure
from bokeh.transform import transform
output_notebook()

list_x = list(df['plannen_totaal'])
list_y = list(df['nieuwbouw_sum'])
desc = list(df['gemeente'])

source = ColumnDataSource(data=dict(x=list_x, y=list_y, desc=desc))
hover = HoverTool(tooltips=[
    ("plannen", "@x"),
    ("nieuwbouw", "@y"),
    ('gemeente', '@desc'),
])

box_zoom = BoxZoomTool()
wheel_zoom = WheelZoomTool()
reset = ResetTool()
undo = UndoTool()
pan = PanTool()

mapper = LinearColorMapper(palette=plasma(256), low=min(list_y), high=max(list_y))

p = figure(plot_width=800, plot_height=600, tools=[hover,box_zoom,wheel_zoom,reset,undo], title="Nieuwbouw versus plannen")
p.circle('x', 'y', size=10, source=source,
         fill_color=transform('y', mapper))


show(p)

In [ ]:
# Waar wordt, volgens de plannen, het meest uitgebreid, even in tabelvorm (top20)?

print(df.sort_values(by='voorraad_plannen_rel', 
      ascending=False)[0:20][['gemeente', 
                              'voorraad_plannen_rel']]\
      .to_string(index=False))

**Analyse** 
* Zowel Ouder-Amstel en Lelystad plannen meer woningen dan ze nu hebben, respectievelijk 1916 en 10025. Klopt dat wel? Is er een fout gemaakt met de data-invoer? Gecheckt: nee. 
* Ook hier veel gemeenten rond Amsterdam. Niet raar omdat we dezelfde twee kolommen met elkaar vergelijken.

In [ ]:
# Waar worden relatief veel nieuwe huizen gepland als je kijkt naar de bouwactiviteit van 2012-2020? 

print(df.sort_values(by='plannen_nieuwbouw_rel_comp' , ascending=False)[0:20][['gemeente', 'plannen_nieuwbouw_rel_comp']]\
      .to_string(index=False))

Laten we de gemeenten nu koppelen aan een shapefile van 2020 en eens kijken hoe dat eruit ziet. 

In [ ]:
shape = gpd.read_file('/content/drive/MyDrive/projects/bouwput_nederland/data/plancapaciteit/shapefiles/cbs_buurt_gemeente_wijk_2020/gemeente_2020_v1.shp')

In [ ]:
shape = shape[shape['H2O'] == 'NEE']

shape = shape[['GM_CODE',
               'GM_NAAM',
               'geometry']]

len(shape)       

In [ ]:
to_check = list(df['gemeentecode'])

In [ ]:
shape[~shape['GM_CODE'].isin(to_check)]

Waar ik al bang voor was: we houden twee gemeenten over die zijn opgegaan in de gemeente Eemsdelta. Als we dit niet oplossen, krijg je een gat in de kaart. 
We kunnen een paar dingen doen:
1. De polygonen van de drie gemeenten (Loppersum, Appingedam en Delfzijl) dissolven, dus tot 1 maken. 
2. De shapefiles als uitgangspunt nemen voor de merge en de ontbrekende waarden van Appingedam en Loppersum kopieren vanuit Delfzijl. Punt daarmee is wel dat als je daarmee gaat rekenen, je dus die waarden driedubbel rekent. Maar deze benadering is makkelijker dan dissolven.
3. Hetzelfde als 2, maar de waarden niet kopieren en het dissolven in Qgis doen. Dat is de makkelijkste oplossing, maar niet herhaalbaar.

In [ ]:
shape[shape['GM_NAAM'] == 'Delfzijl']

In [ ]:
gem = ['Appingedam', 'Loppersum', 'Delfzijl']

to_dissolve = shape[shape['GM_NAAM'].isin(gem)]

to_dissolve = to_dissolve.dissolve()

to_dissolve.at[0,'GM_CODE'] = 'GM0010'
to_dissolve.at[0,'GM_NAAM'] = 'Delfzijl'
to_dissolve

to_dissolve = to_dissolve[['GM_CODE', 'GM_NAAM', 'geometry']]

shape = shape.drop(shape[shape['GM_NAAM'].isin(gem)].index)
shape = shape.append(to_dissolve)

In [ ]:
len(shape)

In [ ]:
merge = pd.merge(shape, df, left_on='GM_CODE', right_on='gemeentecode', how='left')
len(merge)

In [ ]:
merge = merge.drop(merge.index[[342, 343, 344]])

In [ ]:
merge[merge['GM_NAAM'] == 'Beekdaelen']

In [ ]:
len(merge)

In [ ]:
merge = merge.rename(columns = {'gemeentecode': 'gemcode',
  'harde_plannen_totaal': 'pl_hard',
 'zachte_plannen_totaal': 'pl_zacht',
 'plannen_totaal': 'pl_totaal',
 'voorraad_median': 'vr_median',
 'voorraad_min': 'vr_min',
 'voorraad_max': 'vr_max',
 'dif_voorraad': 'vr_dif',
 'nieuwbouw_median': 'nb_median',
 'nieuwbouw_min': 'nb_min',
 'nieuwbouw_max': 'nb_max',
 'nieuwbouw_sum': 'nb_sum',
 'nieuwbouw_perc': 'nb_perc',
 'max_jaar': 'nb_jr_max',
 'min_jaar': 'nb_jr_min',
 'plannen_nieuwbouw_abs': 'pl_nb_abs',
 'plannen_nieuwbouw_rel': 'pl_nb_rel',
 'plannen_nieuwbouw_abs_comp': 'pl_nb_absc',
 'plannen_nieuwbouw_rel_comp': 'pl_nb_relc',
 'voorraad_plannen_abs': 'vr_pl_abs',
 'voorraad_plannen_rel': 'vr_pl_rel'})